In [ ]:
%cd "/data/KunZhao/on-isotropy-of-contrastive-SRL-main/dd++"
# magic code to automatically reload external modules if they change
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import sys
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, LoggingHandler, util, models, evaluation, losses, InputExample
import logging
from datetime import datetime
import gzip
import os
import tarfile
from collections import defaultdict
from torch.utils.data import IterableDataset
import tqdm
from torch.utils.data import Dataset
import random
import pickle
import argparse
from sentence_transformers import evaluation
from sentence_transformers import InputExample
import numpy as np

In [ ]:
!export CUDA_VISIBLE_DEVICES=1

In [ ]:
import transformers.models.bert.modeling_bert

In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
!conda list


In [ ]:
with open ("train.json", "r") as f:
    lines = f.readlines()
print(type(lines))

In [ ]:
print(len(lines))

In [ ]:
# lines_len = []
train_examples = []
for line in lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        for j, n in enumerate(sample["adversarial_negative_responses"]):
            neg = "[CLS] " + n + " [SEP]"
            train_examples.append([context, pos, neg])
print(sample)
print(train_examples[-1])
                            
    # lines_len.append(len(ss.strip()))
# print("context: ")
# print(sample["context"])
# print("\n")
# print("positive_responses:")
# for pos in sample["positive_responses"]:
#     print(pos)

# print("\n")
# print("adversarial_negative_responses:")
# for adv in sample["adversarial_negative_responses"]:
#     print(adv)

# print("\n")
# print("random_negative_responses:")
# for ran in sample["random_negative_responses"]:
#     print(ran)

In [ ]:
c = 0

for l in lines_len:
    if l > 512:
       c = c + 1 
print(c, len(lines_len))

In [ ]:
ss = "[CLS] "
for c in sample["context"]:
    ss = ss + c + " [SEP] "
print(ss.strip())

In [ ]:
train_context = 

In [ ]:
from sentence_transformers import InputExample

train_examples = []
for line in lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        for j, n in enumerate(sample["adversarial_negative_responses"]):
            neg = "[CLS] " + n + " [SEP]"
            train_examples.append(InputExample(texts=[context, pos, neg]))
# print(sample)
# print(train_examples[-1])

In [ ]:
print(len(train_examples))

In [ ]:
print(train_examples[0])
print(train_examples[1])
print(train_examples[2])
print(train_examples[3])

In [ ]:
"""
This examples show how to train a Bi-Encoder for the MS Marco dataset (https://github.com/microsoft/MSMARCO-Passage-Ranking).

The queries and passages are passed independently to the transformer network to produce fixed sized embeddings.
These embeddings can then be compared using cosine-similarity to find matching passages for a given query.

For training, we use MultipleNegativesRankingLoss. There, we pass triplets in the format:
(query, positive_passage, negative_passage)

Negative passage are hard negative examples, that were mined using different dense embedding methods and lexical search methods.
Each positive and negative passage comes with a score from a Cross-Encoder. This allows denoising, i.e. removing false negative
passages that are actually relevant for the query.

With a distilbert-base-uncased model, it should achieve a performance of about 33.79 MRR@10 on the MSMARCO Passages Dev-Corpus

Running this script:
python train_bi-encoder-v3.py
"""
import sys
import json
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, LoggingHandler, util, models, evaluation, losses, InputExample
import logging
from datetime import datetime
import gzip
import os
import tarfile
from collections import defaultdict
from torch.utils.data import IterableDataset
import tqdm
from torch.utils.data import Dataset
import random
import pickle
import argparse
from transformers import AutoTokenizer, DistilBertForSequenceClassification, DistilBertModel

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout


# parser = argparse.ArgumentParser()
# parser.add_argument("--train_batch_size", default=64, type=int)
# parser.add_argument("--max_seq_length", default=512, type=int)
# parser.add_argument("--model_name", required=True)
# parser.add_argument("--max_passages", default=0, type=int)
# parser.add_argument("--epochs", default=10, type=int)
# parser.add_argument("--pooling", default="mean")
# parser.add_argument("--negs_to_use", default=None, help="From which systems should negatives be used? Multiple systems seperated by comma. None = all")
# parser.add_argument("--warmup_steps", default=1000, type=int)
# parser.add_argument("--lr", default=2e-5, type=float)
# parser.add_argument("--num_negs_per_system", default=5, type=int)
# parser.add_argument("--use_pre_trained_model", default=False, action="store_true")
# parser.add_argument("--use_all_queries", default=False, action="store_true")
# parser.add_argument("--ce_score_margin", default=3.0, type=float)
# args = parser.parse_args()

# print(args)

# The  model we want to fine-tune
# model_name = args.model_name

# train_batch_size = args.train_batch_size           #Increasing the train batch size improves the model performance, but requires more GPU memory
# max_seq_length = args.max_seq_length            #Max length for passages. Increasing it, requires more GPU memory
# ce_score_margin = args.ce_score_margin             #Margin for the CrossEncoder score between negative and positive passages
# num_negs_per_system = args.num_negs_per_system         # We used different systems to mine hard negatives. Number of hard negatives to add from each system
# num_epochs = args.epochs                 # Number of epochs we want to train
model_name = "distilbert-base-uncased"
# Distill_classical_model = DistilBertModel.from_pretrained("distilbert-base-uncased")
# state_dict = Distill_classical_model.state_dict()

train_batch_size = 1           #Increasing the train batch size improves the model performance, but requires more GPU memory
max_seq_length = 512            #Max length for passages. Increasing it, requires more GPU memory
# ce_score_margin = args.ce_score_margin             #Margin for the CrossEncoder score between negative and positive passages
# num_negs_per_system = args.num_negs_per_system         # We used different systems to mine hard negatives. Number of hard negatives to add from each system
num_epochs = 10                 # Number of epochs we want to train

use_pre_trained_model = True
# Load our embedding model
if use_pre_trained_model:
    logging.info("use pretrained SBERT model")
    model = SentenceTransformer(model_name)
    model = model.load_state_dict(state_dict, )
    model.max_seq_length = max_seq_length
else:
    logging.info("Create new SBERT model")
    word_embedding_model = models.Transformer(model_name, max_seq_length=max_seq_length)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), args.pooling)
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

model_save_path = 'output/train_bi-encoder-mnrl-{}-{}-pretrain-classical'.format(model_name.replace("/", "-"),  datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))


### Now we read the MS Marco dataset
# data_folder = 'msmarco-data'

# For training the SentenceTransformer model, we need a dataset, a dataloader, and a loss used for training.
# train_dataset = MSMARCODataset(train_queries, corpus=corpus)
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.MultipleNegativesRankingLoss(model=model)

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=1000,
          use_amp=True,
          checkpoint_path=model_save_path,
          checkpoint_save_steps=len(train_dataloader),
          optimizer_params = {'lr': 2e-5},
          )

# Save the model
# model.save(model_save_path)

In [ ]:
from sentence_transformers import evaluation

In [ ]:
print(range(1))

In [ ]:
with open ("dev.json", "r") as f:
    dev_lines = f.readlines()
dev_examples = []
context_list = []
pos_list = []
neg_list = []
for line in dev_lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        for j, n in enumerate(sample["adversarial_negative_responses"]):
            context_list.append(context)
            pos_list.append(pos)
            neg = "[CLS] " + n + " [SEP]"
            neg_list.append(neg)
            dev_examples.append(InputExample(texts=[context, pos, neg]))
    

In [ ]:
with open ("dev.json", "r") as f:
    dev_lines = f.readlines()
dev_examples = []
context_list = []
pos_list = []
neg_list = []
for line in lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        for j, n in enumerate(sample["random_negative_responses"]):
            context_list.append(context)
            pos_list.append(pos)
            neg = "[CLS] " + n + " [SEP]"
            neg_list.append(neg)
            dev_examples.append(InputExample(texts=[context, pos, neg]))

In [ ]:
print(context,pos,neg)

In [ ]:
with open ("test.json", "r") as f:
    dev_lines = f.readlines()
dev_examples = []
context_list = []
pos_list = []
neg_list = []
for line in dev_lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        for j, n in enumerate(sample["adversarial_negative_responses"]):
            context_list.append(context)
            pos_list.append(pos)
            neg = "[CLS] " + n + " [SEP]"
            neg_list.append(neg)
            dev_examples.append(InputExample(texts=[context, pos, neg]))
evaluator = evaluation.TripletEvaluator(context_list, pos_list, neg_list, 0)
model = SentenceTransformer("output/train_bi-encoder-mnrl-bert-base-uncased-2023-08-29_15-22-44/86808")
evaluator(model)

In [ ]:
with open ("test.json", "r") as f:
    dev_lines = f.readlines()
dev_examples = []
context_list = []
pos_list = []
neg_list = []
for line in dev_lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        for j, n in enumerate(sample["adversarial_negative_responses"]):
            context_list.append(context)
            pos_list.append(pos)
            neg = "[CLS] " + n + " [SEP]"
            neg_list.append(neg)
            dev_examples.append(InputExample(texts=[context, pos, neg]))
evaluator = evaluation.TripletEvaluator(context_list, pos_list, neg_list, 0)
model = SentenceTransformer("output/train_bi-encoder-mnrl-random-bert-base-uncased-2023-08-29_15-26-52")
evaluator(model)

In [ ]:
# pos+adv训练，pos+random测试
with open ("test.json", "r") as f:
    dev_lines = f.readlines()
dev_examples = []
context_list = []
pos_list = []
neg_list = []
for line in dev_lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        for j, n in enumerate(sample["random_negative_responses"]):
            context_list.append(context)
            pos_list.append(pos)
            neg = "[CLS] " + n + " [SEP]"
            neg_list.append(neg)
            dev_examples.append(InputExample(texts=[context, pos, neg]))
evaluator = evaluation.TripletEvaluator(context_list, pos_list, neg_list, 0)
model = SentenceTransformer("output/train_bi-encoder-mnrl-bert-base-uncased-2023-08-29_15-22-44/86808")
evaluator(model)

In [ ]:
with open ("test.json", "r") as f:
    dev_lines = f.readlines()
dev_examples = []
context_list = []
pos_list = []
neg_list = []
for line in dev_lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        for j, n in enumerate(sample["adversarial_negative_responses"]):
            context_list.append(context)
            pos_list.append(pos)
            neg = "[CLS] " + n + " [SEP]"
            neg_list.append(neg)
            dev_examples.append(InputExample(texts=[context, pos, neg]))
evaluator = evaluation.TripletEvaluator(context_list, pos_list, neg_list, 0)
model = SentenceTransformer("output/train_bi-encoder-mnrl-distilbert-base-uncased-2023-08-23_21-32-34/72340")
evaluator(model)

In [ ]:
with open ("test.json", "r") as f:
    dev_lines = f.readlines()
dev_examples = []
context_list = []
pos_list = []
neg_list = []
for line in dev_lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        for j, n in enumerate(sample["random_negative_responses"]):
            context_list.append(context)
            pos_list.append(pos)
            neg = "[CLS] " + n + " [SEP]"
            neg_list.append(neg)
            dev_examples.append(InputExample(texts=[context, pos, neg]))
evaluator = evaluation.TripletEvaluator(context_list, pos_list, neg_list, 0)
model = SentenceTransformer("output/train_bi-encoder-mnrl-distilbert-base-uncased-2023-08-23_21-32-34/72340")
evaluator(model)

In [ ]:
with open ("test.json", "r") as f:
    dev_lines = f.readlines()
dev_examples = []
context_list = []
pos_list = []
neg_list = []
for line in dev_lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        for j, n in enumerate(sample["adversarial_negative_responses"]):
            context_list.append(context)
            pos_list.append(pos)
            neg = "[CLS] " + n + " [SEP]"
            neg_list.append(neg)
            dev_examples.append(InputExample(texts=[context, pos, neg]))
evaluator = evaluation.TripletEvaluator(context_list, pos_list, neg_list, 0)
model = SentenceTransformer("output/train_bi-encoder-mnrl-random-distilbert-base-uncased-2023-08-24_10-25-55/72340")
evaluator(model)

In [ ]:
with open ("test.json", "r") as f:
    dev_lines = f.readlines()
dev_examples = []
context_list = []
pos_list = []
neg_list = []
for line in dev_lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        for j, n in enumerate(sample["adversarial_negative_responses"]):
            context_list.append(context)
            pos_list.append(pos)
            neg = "[CLS] " + n + " [SEP]"
            neg_list.append(neg)
            dev_examples.append(InputExample(texts=[context, pos, neg]))
evaluator = evaluation.TripletEvaluator(context_list, pos_list, neg_list, 0)
model = SentenceTransformer("output/train_bi-encoder-mnrl-sentence-t5-base-2023-08-30_11-35-59")
evaluator(model)

In [ ]:
with open ("test.json", "r") as f:
    dev_lines = f.readlines()
dev_examples = []
context_list = []
pos_list = []
neg_list = []
for line in dev_lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        for j, n in enumerate(sample["adversarial_negative_responses"]):
            context_list.append(context)
            pos_list.append(pos)
            neg = "[CLS] " + n + " [SEP]"
            neg_list.append(neg)
            dev_examples.append(InputExample(texts=[context, pos, neg]))
evaluator = evaluation.TripletEvaluator(context_list, pos_list, neg_list, 0)
model = SentenceTransformer("output/train_bi-encoder-mnrl-random-sentence-t5-base-2023-08-30_11-36-59")
evaluator(model)

In [ ]:
with open ("test.json", "r") as f:
    dev_lines = f.readlines()
dev_examples = []
context_list = []
pos_list = []
neg_list = []
line = dev_lines[5]
y_list = []

# print(line)
sample=json.loads(line)
ss = "[CLS] "
for c in sample["context"]:
    ss = ss + c + " [SEP] "
context = ss.strip()
y_list.append(0)
for i, p in enumerate(sample["positive_responses"]):
    pos = "[CLS] " + p + " [SEP]"
    pos_list.append(pos)
    y_list.append(1)
for j, n in enumerate(sample["adversarial_negative_responses"]):
    neg = "[CLS] " + n + " [SEP]"
    neg_list.append(neg)
    y_list.append(2)


In [ ]:
print("context:")
print(context)
print("\n")

print("pos:")
for i, pos in enumerate(pos_list):
    print(i, pos)
print("\n")

print("neg:")
for i, neg in enumerate(neg_list):
    print(i, neg)

In [ ]:
model = SentenceTransformer("output/train_bi-encoder-mnrl-distilbert-base-uncased-2023-08-23_21-32-34/72340")

In [ ]:
context_embedding = model.encode(context)
print(context_embedding.shape)
x_list = []
x_list.append(context_embedding)
for pos in pos_list:
    pos_embedding = model.encode(pos)
    x_list.append(pos_embedding)
for neg in neg_list:
    neg_embedding = model.encode(neg)
    x_list.append(neg_embedding)


In [ ]:
print(len(x_list), len(y_list))

In [ ]:
import numpy as np
x = np.array(x_list)
y = np.array(y_list)
print(len(x), len(y))

In [ ]:
import json
import numpy as np
from openTSNE import TSNE
from openTSNE.affinity import PerplexityBasedNN
from openTSNE import initialization
# from openTSNE.callbacks import ErrorLogger
import utils
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
tsne = TSNE(
    perplexity=50,
    n_iter=200,
    metric="cosine",
    n_jobs=8,
    random_state=0,
)
embedding = tsne.fit(x)

In [ ]:
x = embedding
fig, ax = plt.subplots()


classes = np.unique(y)

default_colors = matplotlib.rcParams["axes.prop_cycle"]
colors = {k: v["color"] for k, v in zip(classes, default_colors())}

point_colors = list(map(colors.get, y))

ax.scatter(x[:, 0], x[:, 1], c=point_colors, marker="v", s=50)

legend_handles = [
    matplotlib.lines.Line2D(
        [],
        [],
        marker="s",
        color="w",
        markerfacecolor=colors[yi],
        ms=10,
        alpha=1,
        linewidth=0,
        markeredgecolor="k",
#         label=yi
    )
    for yi in classes
]
legend_kwargs_ = dict(labels=["context", "pos", "neg"],bbox_to_anchor=(0.32, 1.18), frameon=False, fontsize=10)
ax.legend(handles=legend_handles, **legend_kwargs_)
ax.set_title("example")
fig.savefig('dd_similar.png',bbox_inches="tight")

In [ ]:
from sklearn.metrics.pairwise import paired_cosine_distances, paired_euclidean_distances, paired_manhattan_distances

In [ ]:
with open ("test.json", "r") as f:
    dev_lines = f.readlines()
model = SentenceTransformer("output/train_bi-encoder-mnrl-distilbert-base-uncased-2023-08-23_21-32-34/72340")
dev_examples = []
context_list = []
pos_list = []
neg_list = []
for line in dev_lines:
    # y_list = []
    
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    # y_list.append(0)
    pos_dis_list = []
    context_embedding = model.encode(context, convert_to_numpy=True)
    context_embedding = context_embedding.reshape(1,-1)
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        pos_embedding = model.encode(pos, convert_to_numpy=True)
        pos_embedding = pos_embedding.reshape(1,-1)
        pos_cos_distance = paired_cosine_distances(context_embedding, pos_embedding)
        pos_dis_list.append(pos_cos_distance)
    
    neg_dis_list = []
    for j, n in enumerate(sample["random_negative_responses"]):
        neg = "[CLS] " + n + " [SEP]"
        neg_embedding = model.encode(neg, convert_to_numpy=True)
        neg_embedding = neg_embedding.reshape(1,-1)
        neg_cos_distance = paired_cosine_distances(context_embedding, neg_embedding)
        neg_dis_list.append(neg_cos_distance)
    pos_list.append(pos_dis_list)
    neg_list.append(neg_dis_list)
    

In [ ]:
total_pos_mean_list = []
total_neg_mean_list = []
for i, pos_dis in enumerate(pos_list):
    dis_sum = sum(pos_dis)
    total_pos_mean_list.append(dis_sum/len(pos_dis))

for i, neg_dis in enumerate(neg_list):
    dis_sum = sum(neg_dis)
    total_neg_mean_list.append(dis_sum/len(neg_dis))

In [ ]:
print(total_pos_mean_list)

In [ ]:
print(total_neg_mean_list)

In [ ]:
print(total_neg_mean_list[0][0])

In [ ]:
count = 0
for dis in total_pos_mean_list:
    if dis[0] > 0.9:
        count = count + 1
print(count, len(total_pos_mean_list))

In [ ]:
count = 0
for dis in total_neg_mean_list:
    if dis[0] < 0.9:
        count = count + 1
print(count, len(total_neg_mean_list))

In [ ]:
count = 0
n = 0
for i, pos_dis in enumerate(pos_list):
    for dis in pos_dis:
        n = n + 1
        if dis[0] > 1.2:
            count = count + 1

for i, neg_dis in enumerate(neg_list):
    for dis in neg_dis:
        n = n + 1
        if dis[0] < 0.3:
            count = count + 1
print(count, n)

In [ ]:
5/11420

In [ ]:
count = 0
n = 0
for i, pos_dis in enumerate(pos_list):
    for neg_dis in neg_list[i]:
        if neg[0]

In [ ]:
from enum import Enum
from typing import Iterable, Dict
import torch.nn.functional as F
from torch import nn, Tensor
from sentence_transformers.SentenceTransformer import SentenceTransformer
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
class MLP(nn.Module):
    def __init__(self, hidden_size, num_labels):
        super(MLP, self).__init__()
        self.classifier = nn.Linear(hidden_size, num_labels)
        self.num_labels = num_labels
        
    def forward(self, hidden_states, labels):
        logits = self.classifier(hidden_states)

        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits, labels)
        return logits, loss

In [ ]:
with open ("train.json", "r") as f:
    train_lines = f.readlines()
train_classification_examples = []
context_list = []
pos_list = []
neg_list = []
for line in train_lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        train_classification_examples.append((context, pos, 1))
    for j, n in enumerate(sample["random_negative_responses"]):
        neg = "[CLS] " + n + " [SEP]"
        train_classification_examples.append((context, neg, 0))
model = SentenceTransformer("output/train_bi-encoder-mnrl-distilbert-base-uncased-2023-08-23_21-32-34/72340")
# print(train_classification_examples)

In [ ]:
with open ("test.json", "r") as f:
    test_lines = f.readlines()
test_classification_examples = []
context_list = []
pos_list = []
neg_list = []
for line in test_lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        test_classification_examples.append((context, pos, 1))
    for j, n in enumerate(sample["random_negative_responses"]):
        neg = "[CLS] " + n + " [SEP]"
        test_classification_examples.append((context, neg, 0))

In [ ]:
train_classification_dataloader = DataLoader(train_classification_examples, shuffle=True, batch_size=32768)
test_classification_dataloader = DataLoader(test_classification_examples, shuffle=True, batch_size=32768)
num_train_data = len(train_classification_examples)
num_test_data = len(test_classification_examples)

In [ ]:
for data in train_classification_dataloader:
    print(data)
    break

In [ ]:
print(train_classification_examples[0])

In [ ]:
sample = train_classification_examples[0]
with torch.no_grad():
    context_embedding = torch.tensor(model.encode(sample[0]))
    response_embedding = torch.tensor(model.encode(sample[1]))
    print(context_embedding.shape)
    hidden_state = torch.cat([context_embedding, response_embedding])

In [ ]:
print(hidden_state.shape)

In [ ]:
classification_model = MLP(1536, 2).cuda()

In [ ]:
import os
import torch
num_epoch = 30
epoch = 0
load_checkpoint_path = "./random_test/"
if not os.path.exists(load_checkpoint_path):
   os.mkdir(load_checkpoint_path)
optimizer = torch.optim.Adam(classification_model.parameters(), lr=1e-5)
for epoch in range(num_epoch):
    accu = 0
    for train_data in train_classification_dataloader:
        with torch.no_grad():
            context_embedding = torch.tensor(model.encode(train_data[0])).to("cuda")
            response_embedding = torch.tensor(model.encode(train_data[1])).to("cuda")
            # print(context_embedding.shape)
            hidden_state = torch.cat([context_embedding, response_embedding], dim=1)
            # print(hidden_state.shape)
        
        outputs = classification_model(hidden_state, train_data[2].to("cuda"))
        # print(data[2])
        loss = outputs[1]
        logits = outputs[0]
        # print(outputs)
        # print(loss)
        loss.backward()
        optimizer.step()
    total_num_right = 0
    for test_data in test_classification_dataloader:
        with torch.no_grad():
            labels = test_data[2].to("cuda")
            context_embedding = torch.tensor(model.encode(test_data[0])).to("cuda")
            response_embedding = torch.tensor(model.encode(test_data[1])).to("cuda")
            # print(context_embedding.shape)
            hidden_state = torch.cat([context_embedding, response_embedding], dim=1)
            # print(hidden_state.shape)
            outputs = classification_model(hidden_state, labels)
            logits = outputs[0]
            score = logits.argmax(dim=-1)
            num_right = ((score == labels).float()).sum()
            total_num_right += num_right
    right_rate = total_num_right/num_test_data
    print(right_rate)
    if right_rate > accu:
        accu = right_rate
        state = {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "best_accu": accu,
        }
        torch.save(state, load_checkpoint_path + "checkpoint.bin")
    # print(right_rate)

In [ ]:
train_examples = []
for line in lines[:10]:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        for j, n in enumerate(sample["adversarial_negative_responses"]):
            neg = "[CLS] " + n + " [SEP]"
            train_examples.append([context, pos, neg])
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=4)

In [ ]:
for i, data in enumerate(train_dataloader):
    pos_embed = 

In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, InputExample
from torch.utils.data import DataLoader

model = SentenceTransformer("output/train_bi-encoder-mnrl-bert-base-uncased-2023-08-29_15-22-44/86808")
train_examples = [
    InputExample(texts=['This is a positive pair', 'Where the distance will be minimized'], label=1),
    InputExample(texts=['This is a negative pair', 'Their distance will be increased'], label=0)]

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=2)
train_loss = losses.ContrastiveLoss(model=model)


In [ ]:
print(train_dataloader)
data_iterators = iter(train_dataloader)
data = next(data_iterators)

print(data)

In [ ]:
with open ("train.json", "r") as f:
    lines = f.readlines()

train_examples = []
for line in lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        for j, n in enumerate(sample["adversarial_negative_responses"]):
            neg = "[CLS] " + n + " [SEP]"
            train_examples.append((context, pos, neg))
model_encoder = SentenceTransformer("output/train_bi-encoder-mnrl-bert-base-uncased-2023-08-29_15-22-44/86808")
# print(train_classification_examples)

In [ ]:
class SiameseDistanceMetric(Enum):
    """
    The metric for the contrastive loss
    """
    EUCLIDEAN = lambda x, y: F.pairwise_distance(x, y, p=2)
    MANHATTAN = lambda x, y: F.pairwise_distance(x, y, p=1)
    COSINE_DISTANCE = lambda x, y: 1-F.cosine_similarity(x, y, dim=0)


class test_ContrastiveLoss(nn.Module):
    """
    Contrastive loss. Expects as input two texts and a label of either 0 or 1. If the label == 1, then the distance between the
    two embeddings is reduced. If the label == 0, then the distance between the embeddings is increased.

    Further information: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf

    :param model: SentenceTransformer model
    :param distance_metric: Function that returns a distance between two embeddings. The class SiameseDistanceMetric contains pre-defined metrices that can be used
    :param margin: Negative samples (label == 0) should have a distance of at least the margin value.
    :param size_average: Average by the size of the mini-batch.

    Example::

        from sentence_transformers import SentenceTransformer, LoggingHandler, losses, InputExample
        from torch.utils.data import DataLoader

        model = SentenceTransformer('all-MiniLM-L6-v2')
        train_examples = [
            InputExample(texts=['This is a positive pair', 'Where the distance will be minimized'], label=1),
            InputExample(texts=['This is a negative pair', 'Their distance will be increased'], label=0)]

        train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=2)
        train_loss = losses.ContrastiveLoss(model=model)

        model.fit([(train_dataloader, train_loss)], show_progress_bar=True)

    """

    def __init__(self, distance_metric=SiameseDistanceMetric.COSINE_DISTANCE, margin: float = 0.5, size_average:bool = True):
        super(test_ContrastiveLoss, self).__init__()
        self.distance_metric = distance_metric
        self.margin = margin
        self.size_average = size_average

    def get_config_dict(self):
        distance_metric_name = self.distance_metric.__name__
        for name, value in vars(SiameseDistanceMetric).items():
            if value == self.distance_metric:
                distance_metric_name = "SiameseDistanceMetric.{}".format(name)
                break

        return {'distance_metric': distance_metric_name, 'margin': self.margin, 'size_average': self.size_average}

    def forward(self, reps, labels: Tensor):
        # reps = [self.model(sentence_feature)['sentence_embedding'] for sentence_feature in sentence_features]
        # assert len(reps) == 2
        rep_anchor, rep_other = reps.chunk(2,-1)
        print(rep_anchor.shape,rep_other.shape)
        distances = self.distance_metric(rep_anchor, rep_other)
        losses = 0.5 * (labels.float() * distances.pow(2) + (1 - labels).float() * F.relu(self.margin - distances).pow(2))
        return losses.mean() if self.size_average else losses.sum()

In [ ]:
sample = train_examples[0]
print(sample)

In [ ]:
context_embedding = torch.tensor(model_encoder.encode(sample[0]))
pos_response_embedding = torch.tensor(model_encoder.encode(sample[1]))
neg_response_embedding = torch.tensor(model_encoder.encode(sample[2]))

In [ ]:
print(type(model_encoder.encode(sample[0])))

In [ ]:
test_loss = test_ContrastiveLoss()

In [ ]:

loss = test_loss(pos_response_embedding, labels=torch.tensor(1))

In [ ]:
print(loss)

In [ ]:
from 

In [ ]:
print(robust_pos_response_feature.shape, pos_response_embedding.shape)

In [ ]:
from enum import Enum
from typing import Iterable, Dict
import torch.nn.functional as F
from torch import nn, Tensor
from sentence_transformers.SentenceTransformer import SentenceTransformer


class SiameseDistanceMetric(Enum):
    """
    The metric for the contrastive loss
    """
    EUCLIDEAN = lambda x, y: F.pairwise_distance(x, y, p=2)
    MANHATTAN = lambda x, y: F.pairwise_distance(x, y, p=1)
    COSINE_DISTANCE = lambda x, y: 1-F.cosine_similarity(x, y)


class DisContrastiveLoss(nn.Module):
    """
    Contrastive loss. Expects as input two texts and a label of either 0 or 1. If the label == 1, then the distance between the
    two embeddings is reduced. If the label == 0, then the distance between the embeddings is increased.

    Further information: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf

    :param model: SentenceTransformer model
    :param distance_metric: Function that returns a distance between two embeddings. The class SiameseDistanceMetric contains pre-defined metrices that can be used
    :param margin: Negative samples (label == 0) should have a distance of at least the margin value.
    :param size_average: Average by the size of the mini-batch.

    Example::

        from sentence_transformers import SentenceTransformer, LoggingHandler, losses, InputExample
        from torch.utils.data import DataLoader

        model = SentenceTransformer('all-MiniLM-L6-v2')
        train_examples = [
            InputExample(texts=['This is a positive pair', 'Where the distance will be minimized'], label=1),
            InputExample(texts=['This is a negative pair', 'Their distance will be increased'], label=0)]

        train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=2)
        train_loss = losses.ContrastiveLoss(model=model)

        model.fit([(train_dataloader, train_loss)], show_progress_bar=True)

    """
    def __init__(self, distance_metric=SiameseDistanceMetric.COSINE_DISTANCE, margin: float = 0.5, size_average:bool = True):
        super(DisContrastiveLoss, self).__init__()
        self.distance_metric = distance_metric
        self.margin = margin
        # self.model = model
        self.size_average = size_average

    def get_config_dict(self):
        distance_metric_name = self.distance_metric.__name__
        for name, value in vars(SiameseDistanceMetric).items():
            if value == self.distance_metric:
                distance_metric_name = "SiameseDistanceMetric.{}".format(name)
                break

        return {'distance_metric': distance_metric_name, 'margin': self.margin, 'size_average': self.size_average}

    def forward(self, pos_response_feature, neg_response_feature):
        # reps = [self.model(sentence_feature)['sentence_embedding'] for sentence_feature in sentence_features]
        # assert len(reps) == 2
        robust_pos_response_feature, non_robust_pos_response_feature = pos_response_feature.chunk(2,-1)
        robust_neg_response_feature, non_robust_neg_response_feature = neg_response_feature.chunk(2,-1)
        # 使同 一 feature里的robust与non robust feature远离
        pos_feature_distances = self.distance_metric(robust_pos_response_feature, non_robust_pos_response_feature)
        inside_pos_losses = 0.5 * (0.float() * pos_feature_distances.pow(2) + (1 - 0).float() * F.relu(self.margin - pos_feature_distances).pow(2))
        
        neg_feature_distances = self.distance_metric(robust_neg_response_feature, non_robust_neg_response_feature)
        inside_neg_losses = 0.5 * (0.float() * neg_feature_distances.pow(2) + (1 - 0).float() * F.relu(self.margin - neg_feature_distances).pow(2))
        
        #使不同feature的robust接近
        robust_distances = self.distance_metric(robust_neg_response_feature, robust_neg_response_feature)
        
        robust_losses = 0.5 * (labels.float() * robust_distances.pow(2) + (1 - labels).float() * F.relu(self.margin - robust_distances).pow(2))
        non_robust_losses = 0.5 * (labels.float() * non_robust_distances.pow(2) + (1 - labels).float() * F.relu(self.margin - non_robust_distances).pow(2))
        return losses.mean() if self.size_average else losses.sum()
    

In [ ]:
constrastive_loss = DisContrastiveLoss()
loss = constrastive_loss(pos_response_embedding, neg_response_embedding)


In [ ]:
azime

In [ ]:
!conda list

In [ ]:
import torch
a = torch.ones(32)

In [ ]:
2*a

In [ ]:
# no contrastive learning accuracy
with open ("train.json", "r") as f:
    train_lines = f.readlines()
train_classification_examples = []
context_list = []
pos_list = []
neg_list = []
for line in train_lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        train_classification_examples.append((context, pos, 1))
    for j, n in enumerate(sample["adversarial_negative_responses"]):
        neg = "[CLS] " + n + " [SEP]"
        train_classification_examples.append((context, neg, 0))
        
with open ("test.json", "r") as f:
    test_lines = f.readlines()
test_classification_examples = []
context_list = []
pos_list = []
neg_list = []
for line in test_lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        test_classification_examples.append((context, pos, 1))
    for j, n in enumerate(sample["adversarial_negative_responses"]):
        neg = "[CLS] " + n + " [SEP]"
        test_classification_examples.append((context, neg, 0))
train_classification_dataloader = DataLoader(train_classification_examples, shuffle=True, batch_size=32)
test_classification_dataloader = DataLoader(test_classification_examples, shuffle=True, batch_size=32)
num_train_data = len(train_classification_examples)
num_test_data = len(test_classification_examples)


In [ ]:
for data in train_classification_dataloader:
    print(data)

In [ ]:
import torch
from transformers import AutoTokenizer, DistilBertForSequenceClassification, DistilBertModel
from enum import Enum
from typing import Iterable, Dict
import torch.nn.functional as F
from torch import nn, Tensor
from sentence_transformers.SentenceTransformer import SentenceTransformer
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
class classifier_no_contrastive(nn.Module):
    def __init__(self, hidden_size, num_labels):
        super(classifier_no_contrastive, self).__init__()
        self.encoder = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.classifier = nn.Linear(hidden_size, num_labels)
        self.num_labels = num_labels
        
    def forward(self, data):
        contexts = tokenizer(data[0], return_tensors="pt", padding=True, truncation=True)
        responses = tokenizer(data[1], return_tensors="pt", padding=True)
        context_embeddings = self.encoder(**contexts)[0][:, 0]
        responses_embeddings = self.encoder(**responses)[0][:, 0]
        hidden_states = torch.cat([context_embeddings, responses_embeddings],dim=1)
        labels = data[2]
        # hidden_states = self.encoder(inputs)
        logits = self.classifier(hidden_states)

        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits, labels)
        return logits, loss




# predicted_class_id = logits.argmax().item()

# # To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
# num_labels = len(model.config.id2label)
# model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

# labels = torch.tensor([1])
# loss = model(**inputs, labels=labels).loss

In [ ]:
from transformers import AutoTokenizer, DistilBertForSequenceClassification, DistilBertModel
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")
for data in train_classification_dataloader:
    contexts = tokenizer(data[0], return_tensors="pt", padding=True, truncation=True)
    responses = tokenizer(data[1], return_tensors="pt", padding=True)
    context_embeddings = model(**contexts)[0][:, 0]
    responses_embeddings = model(**responses)[0][:, 0]
    hidden_states = torch.cat([context_embeddings, responses_embeddings],dim=1)
    print(hidden_states.shape)
# with torch.no_grad():
#     logits = model(**inputs).logits

In [ ]:
import os
num_epoch = 30
epoch = 0
load_checkpoint_path = "./pretrain_basic_train_basic/"
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
classification_model_1 = classifier_no_contrastive(1536, 2)
if not os.path.exists(load_checkpoint_path):
   os.mkdir(load_checkpoint_path)
optimizer = torch.optim.Adam(classification_model.parameters(), lr=1e-5)
for epoch in range(num_epoch):
    accu = 0
    for train_data in train_classification_dataloader:
        # with torch.no_grad():
        #     context_embedding = torch.tensor(model.encode(train_data[0]))
        #     response_embedding = torch.tensor(model.encode(train_data[1]))
        #     # print(context_embedding.shape)
        #     hidden_state = torch.cat([context_embedding, response_embedding], dim=1)
        #     # print(hidden_state.shape)

        outputs = classification_model_1(train_data)
        # print(data[2])
        loss = outputs[1]
        logits = outputs[0]
        # print(outputs)
        # print(loss)
        loss.backward()
        optimizer.step()
    total_num_right = 0
    for test_data in test_classification_dataloader:
        with torch.no_grad():
            # labels = test_data[2]
            # context_embedding = torch.tensor(model.encode(test_data[0]))
            # response_embedding = torch.tensor(model.encode(test_data[1]))
            # # print(context_embedding.shape)
            # hidden_state = torch.cat([context_embedding, response_embedding], dim=1)
            # print(hidden_state.shape)
            outputs = classification_model_1(test_data)
            logits = outputs[0]
            score = logits.argmax(dim=-1)
            num_right = ((score == labels).float()).sum()
            total_num_right += num_right
    right_rate = total_num_right/num_test_data
    print(right_rate)
    if right_rate > accu:
        accu = right_rate
        state = {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "best_accu": accu,
        }
        torch.save(state, load_checkpoint_path + "checkpoint.bin")

In [ ]:
import random
with open ("test.json", "r") as f:
    test_lines = f.readlines()
test_classification_examples = []
context_list = []
pos_list = []
neg_list = []
for line in test_lines:
    # print(line)
    sample=json.loads(line)
    ss = "[CLS] "
    for c in sample["context"]:
        ss = ss + c + " [SEP] "
    context = ss.strip()
    for i, p in enumerate(sample["positive_responses"]):
        pos = "[CLS] " + p + " [SEP]"
        test_classification_examples.append((context, pos, 1))
    for j, n in enumerate(sample["adversarial_negative_responses"]):
        neg = "[CLS] " + n + " [SEP]"
        test_classification_examples.append((context, neg, 0))
random.shuffle(test_classification_examples)

In [ ]:
json_samples = json.dumps(test_classification_examples, ensure_ascii=False, indent=2)
with open("testset.json", 'w', encoding='utf-8') as f:
    f.write(json_samples)